# Technical Question Explainer Tool

This notebook implements a tool that uses OpenAI's API to provide explanations for technical questions.

## Setup and Imports

In [1]:
# imports
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI


This section imports the necessary libraries:
- `os`: For environment variable handling
- `requests`: For making HTTP requests
- `dotenv`: For loading environment variables from a .env file
- `IPython.display`: For displaying markdown in Jupyter notebooks
- `openai`: The OpenAI API client

## Environment Setup and API Key Validation

In [2]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# validate API key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


**This section:**
1. Loads environment variables from the .env file
2. Retrieves the OpenAI API key
3. Validates the API key format and presence
4. Provides helpful error messages for common issues

## Constants and Client Initialization

In [4]:
# constants
MODEL_GPT = 'gpt-4o-mini'
openai = OpenAI()

In [5]:
def get_streaming_response(
    prompt: str,
    system_prompt: str,
    model: str = "gpt-4o-mini",
    client: openai = None
) -> str:
    """
    Generate a streaming response from OpenAI's API and display it as Markdown.
    
    Args:
        prompt (str): The user's prompt/question to be sent to the API
        system_prompt (str): The system prompt that sets the context for the model
        model (str, optional): The OpenAI model to use. Defaults to "gpt-4"
        client (Optional[OpenAI], optional): An OpenAI client instance. If None, creates a new one.
    
    Returns:
        str: The accumulated response from the API
        
    Example:
        >>> system_prompt = "You're an experienced programmer."
        >>> user_prompt = "Explain what this code does: print('Hello, World!')"
        >>> response = get_streaming_response(user_prompt, system_prompt)
    """
    # Initialize OpenAI client if not provided
    if client is None:
        client = OpenAI()
    
    # Create messages array
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    
    # Create streaming response
    response_stream = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True
    )
    
    # Accumulate and display response
    accumulated_response = ""
    
    for chunk in response_stream:
        if chunk.choices:
            content = chunk.choices[0].delta.content
            if content:
                accumulated_response += content
    
    # Display the markdown response
    display(Markdown(accumulated_response))
    
    return accumulated_response


In [6]:
# Create explainer instance
explainer = CodeExplainer(model="gpt-4o-mini")

# Your code to explain
code = """yield from {book.get("author") for book in books if book.get("author")}"""

# Get explanation
explanation = explainer.explain_code(code)

NameError: name 'CodeExplainer' is not defined

**This section:**
1. Defines the GPT model to be used
2. Initializes the OpenAI client

## System Prompt Definition

In [29]:
system_prompt = "You're experienced programmer, specialized in python code \
I going to give to you a piece of code and you need to explain it to me."

This defines the system prompt that sets the context for the GPT model. The prompt instructs the model to act as an experienced Python programmer who will explain code.

## Sample Code for Analysis


In [40]:
code = """yield from {book.get("author") for book in books if book.get("author")}"""


This section defines the sample Python code that will be explained by the model.

## Message Construction


In [34]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": code}
]

This constructs the message array for the OpenAI API call, including:
1. The system message that sets the context
2. The user message containing the code to be explained

## Basic API Call Example

In [35]:
# To give you a preview -- calling OpenAI with system and user messages:
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

The code `yield from {book.get("author") for book in books if book.get("author")}` does several things:

1. **Set Comprehension**: The expression `{book.get("author") for book in books if book.get("author")}` creates a set. A set in Python is a collection type that automatically removes duplicate values. 
   - `book.get("author")` attempts to retrieve the value associated with the key `"author"` from each `book` dictionary in the `books` iterable.
   - The `if book.get("author")` condition filters out any books that do not have an author (i.e., where `get("author")` returns `None` or an empty string).

2. **Yielding Values**: The `yield from` statement is used within a generator function. It allows the generator to yield all values from another iterable (in this case, the set created by the set comprehension) one by one.
   - This means that the generator function will return each author in the set, effectively producing them on-the-fly without creating a list or other data structure i

This demonstrates a basic API call to OpenAI:
1. Creates a chat completion request
2. Prints the response content from the first choice

## Streaming Response Implementation

In [41]:
# Create a streaming response generator
response_stream = openai.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": code}
    ],
    stream=True  # Enable streaming
)

accumulated_response = ""

# Read streaming chunks
for chunk in response_stream:
    if chunk.choices:
        content = chunk.choices[0].delta.content
        if content:
            accumulated_response += content

display(Markdown(accumulated_response))


The line of code you've posted uses a generator function and comprehensions in Python. It employs the `yield from` syntax, which is typically used within generator functions to yield all values from an iterable.

Let's break down the statement you have and then provide some context and example usage.

### Breakdown of the Code

- **Set Comprehension**: `{book.get("author") for book in books if book.get("author")}` is creating a set of authors from a list of books. It iterates through each `book` in the `books` iterable (which is expected to be a list or similar structure), calling `get("author")` on each `book` dictionary to extract the author. Only authors that exist (i.e., are not `None` or falsy) will be included in the set.
  
- **`yield from`**: This statement allows the function to yield each element from the iterable returned by the set comprehension. The result would be that each author will be yielded one by one.

### Example Usage

Now let's assume you have the following code to utilize this:

```python
def get_authors(books):
    # Yielding each unique author from the books
    yield from {book.get("author") for book in books if book.get("author")}

# Example list of books
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": None},
    {"title": "Book 4", "author": "Author A"},
]

# Using the generator
for author in get_authors(books):
    print(author)
```

### Output
This would output:
```
Author A
Author B
```

### Explanation
- The `get_authors` function extracts authors from a list of book dictionaries without returning duplicates (because it uses a set).
- When you iterate over `get_authors(books)`, it will yield each unique author found in the books.

### Notes
1. **Generality**: This approach works well for extracting unique items from a collection.
2. **Efficiency**: Using a generator here is memory efficient, especially with large datasets since it does not require the entire list of authors to be stored in memory.
3. **Duplicates**: The set ensures that there are no duplicate authors in the output. If you want to retain duplicates while still yielding values, you would not use a set.

Feel free to modify the function and the logic inside it based on your specific requirements!

This section implements streaming responses:
1. Creates a streaming chat completion request
2. Accumulates the response chunks
3. Displays the final result as markdown